In [1]:
!pip install opencv-python
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tensorflow.keras import models, layers, activations, \
optimizers, utils, losses, initializers, metrics, callbacks
import os
import cv2 as cv

In [6]:
folder_path = 'testing'
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = f'checkpoints/{folder_path}/test_model5.keras'
exists = os.path.exists(model_path)

# Carrega modelo se já existir um checkpoint, caso contrário, o cria.
if exists:
    model = models.load_model(model_path)
else: 
    model = models.Sequential([
        layers.Resizing(32, 32),
        layers.Rescaling(1.0/255),
        layers.BatchNormalization(axis=1),
        layers.RandomRotation((-0.05, 0.05)),
        layers.Conv2D(32, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
            ),
        # layers.MaxPooling2D((2, 2)),
        layers.Conv2D(48, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        # layers.MaxPooling2D((2, 2)), # tirar dps
        layers.Conv2D(32, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        # layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128,
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        layers.Dense(80,
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        layers.Dense(64,
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        layers.Dense(62,
            activation = 'sigmoid',
            kernel_initializer = initializers.RandomNormal()
        )
        ])

if exists:
    model.summary()
else:
    model.compile(
    optimizer = optimizers.Adam(
    learning_rate = learning_rate
),
    loss = losses.SparseCategoricalCrossentropy(), # trocar 
    metrics = [ 'accuracy' ]
)
    
train = utils.image_dataset_from_directory(
    "organizedData",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (1200, 900),
    batch_size= batch_size
    )

test = utils.image_dataset_from_directory(
    "organizedData",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (1200, 900),
    batch_size= batch_size
    )

model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        ),
        # callbacks.LearningRateScheduler(
        #     lambda epoch, lr : 1e-3 * 10 ** -(epoch / 20)
        # )
    ]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 32, 32, 3)         0         
                                                                 
 rescaling_1 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 3)         128       
 chNormalization)                                                
                                                                 
 random_rotation_1 (RandomR  (None, 32, 32, 3)         0         
 otation)                                                        
                                                                 
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 48)       

OSError: [Errno 22] Invalid argument: 'checkpoints/testing\\test_model5.keras'